In [ ]:
import os
# folder_name = 'Aryansh/blood_pressure'


# os.makedirs(folder_name, exist_ok=True)
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [ ]:
# # prompt: mount drive

# from google.colab import drive
# drive.mount('/content/drive')


#Preprocessing PPG AND ECG SIGNAL

In [ ]:
def extract_ppg_features(ppg_array, fs=100):
    """
    Extracts PPG waveform features for each row in the input array.

    Parameters:
        ppg_array (numpy array): PPG signal data of shape (n_samples, n_timepoints).
        fs (int): Sampling frequency (default: 100 Hz).

    Returns:
        numpy array: Extracted feature matrix of shape (n_samples, 9).
    """
    num_samples, _ = ppg_array.shape
    feature_list = []

    for i in range(num_samples):
        ppg_signal = ppg_array[i, :]
        ppg_signal = (ppg_signal - np.min(ppg_signal)) / (np.max(ppg_signal) - np.min(ppg_signal))  # Normalize

        # Find systolic peaks and diastolic valleys
        peaks, _ = find_peaks(ppg_signal, distance=50)
        valleys, _ = find_peaks(-ppg_signal, distance=50)

        if len(peaks) < 2 or len(valleys) < 2:
            feature_list.append([np.nan] * 9)  # Append NaN for invalid samples
            continue

        PPGp = ppg_signal[peaks[0]]  # First systolic peak
        PPGv1 = ppg_signal[valleys[0]]  # First diastolic valley

        # Compute time intervals
        T1 = peaks[0] - valleys[0]
        T2 = valleys[1] - peaks[0] if len(valleys) > 1 else T1
        T3 = peaks[1] - valleys[1] if len(peaks) > 1 and len(valleys) > 1 else T1
        T4 = valleys[0] - peaks[0]
        T5 = peaks[0] - valleys[0]

        # Compute features
        ETR = (T4 + T5) / (T1 + T2 + T3) if (T1 + T2 + T3) != 0 else np.nan
        BD = T1 + T2 + T3  # Beat Duration
        HR = BD / 60  # Heart Rate
        SVC = (PPGp - PPGv1) / T1 if T1 != 0 else np.nan  # Systolic Volume Change
        DVC = (PPGp - PPGv1) / T2 if T2 != 0 else np.nan  # Diastolic Volume Change

        # First derivative for systolic & diastolic max volume change
        diff_signal = np.diff(ppg_signal)
        systolic_max_volume_change = np.max(diff_signal)
        diastolic_max_volume_change = np.min(diff_signal)

        # Pulse Wave Intensity Ratio
        pulse_wave_intensity_ratio = PPGp / PPGv1 if PPGv1 != 0 else np.nan

        # Pulse Wave Arrival Time (Converted to seconds)
        pulse_wave_arrival_time = T1 / fs

        # Store extracted features
        features = [
            ETR, BD, HR, SVC, DVC,
            systolic_max_volume_change, diastolic_max_volume_change,
            pulse_wave_intensity_ratio, pulse_wave_arrival_time
        ]
        feature_list.append(features)

    return np.array(feature_list)

In [ ]:
#extracting time features
def extract_time_domain_features(signal):
    mean = np.mean(signal, axis=1)
    std_dev = np.std(signal, axis=1)
    maximum = np.max(signal, axis=1)
    minimum = np.min(signal, axis=1)
    rng = maximum - minimum
    skewness = stats.skew(signal, axis=1)
    kurtosis = stats.kurtosis(signal, axis=1)
    entropy = stats.entropy(signal, axis=1)
    #features
    features = np.column_stack((mean, std_dev, maximum, minimum, rng, skewness, kurtosis,entropy))
    return features

In [ ]:
#extracting fre features
def extract_frequency_domain_features(signal, fs=125):
    freqs, psd = welch(signal, fs=fs, nperseg=256, axis=1)
    dominant_freqs = freqs[np.argmax(psd, axis=1)]
    power = np.sum(psd, axis=1)
    features = np.column_stack((dominant_freqs, power))
    return features

In [ ]:
#extracting special ppg features
def extract_ppg_specific_features(ppg_signal, fs=125):
    pulse_intervals = []
    pulse_amplitudes = []
    for signal in ppg_signal:
        peaks, _ = find_peaks(signal, distance=fs*0.5)
        if len(peaks) > 1:
            pulse_intervals_signal = np.diff(peaks) / fs
            pulse_amplitudes_signal = signal[peaks]
            pulse_intervals.append(np.mean(pulse_intervals_signal))
            pulse_amplitudes.append(np.mean(pulse_amplitudes_signal))
        else:
            pulse_intervals.append(np.nan)
            pulse_amplitudes.append(np.nan)
    features = np.column_stack((pulse_intervals, pulse_amplitudes))
    return features

In [ ]:
def extract_ecg_specific_features(ecg_signal, fs=125):
    rr_intervals = []
    hrv_metrics = []
    for signal in ecg_signal:
        peaks, _ = find_peaks(signal, distance=fs*0.6)
        if len(peaks) > 1:
            rr_intervals_signal = np.diff(peaks) / fs
            rr_intervals.append(np.mean(rr_intervals_signal))
            hrv_metrics.append(np.std(rr_intervals_signal))
        else:
            rr_intervals.append(np.nan)
            hrv_metrics.append(np.nan)
    features = np.column_stack((rr_intervals, hrv_metrics))
    return features

In [ ]:
import scipy.stats as stats
from scipy.signal import welch, find_peaks

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
# # # prompt: unzip BTP2

# !unzip /content/Aryansh.zip
# # #

#Reading the Normalized data

In [ ]:
ppg_data = pd.read_csv("/content/BTP-6th sem/PPGn.csv")
ecg_data = pd.read_csv("/content/BTP-6th sem/ECGn.csv")

In [ ]:
ppg_data.head()

,0.284202899,0.281842307,0.277429883,0.270163054,0.260293427,0.24890253,0.237272346,0.226235133,0.215870008,0.20565962,...,0.724907285,0.798516655,0.854479092,0.894856726,0.922488458,0.939926678,0.94879792,0.950107267,0.944323716,0.931834752
0,0.072503,0.061680,0.049863,0.035688,0.020620,0.009029,0.007230,0.021855,0.058054,0.118052,...,0.262787,0.251111,0.240562,0.230514,0.220351,0.209732,0.198686,0.187115,0.175173,0.163187
1,0.595704,0.571820,0.544320,0.509617,0.467623,0.421375,0.375406,0.333788,0.298807,0.270736,...,0.719616,0.811132,0.882430,0.933363,0.966643,0.986496,0.996660,1.000000,0.997799,0.989873
2,0.551805,0.555711,0.559309,0.559208,0.553901,0.543614,0.529526,0.512863,0.494343,0.474143,...,0.188682,0.184006,0.180040,0.175960,0.171793,0.166963,0.161662,0.155741,0.149806,0.143589
3,0.396400,0.369185,0.343201,0.317870,0.293623,0.271624,0.253089,0.238599,0.227789,0.219520,...,0.061957,0.056183,0.058445,0.073496,0.105619,0.157776,0.230214,0.320582,0.423281,0.530430
4,0.115908,0.115054,0.111783,0.105170,0.095507,0.084024,0.072121,0.060599,0.049361,0.037795,...,0.238316,0.226023,0.215098,0.205366,0.195831,0.186683,0.177409,0.167738,0.157610,0.147356


In [ ]:
ecg_data.head()

,0.033508398,0.019684831,0.01997061,0.041769538,0.083304533,0.135552932,0.187353922,0.230445582,0.26180281,0.282779855,...,0.301179836,0.113993808,0.06916452,0.126250592,0.202210051,0.236913809,0.224647503,0.197706314,0.187422906,0.19945276
0,0.317213,0.291129,0.277450,0.270884,0.249435,0.200533,0.148526,0.153048,0.267678,0.485348,...,0.116989,0.157707,0.193901,0.221306,0.238786,0.248691,0.254827,0.259577,0.263116,0.264948
1,0.185798,0.192198,0.195500,0.194166,0.187509,0.175102,0.156220,0.130252,0.098237,0.064406,...,0.512992,0.236748,0.085207,0.078368,0.151941,0.220430,0.238343,0.215953,0.190369,0.186703
2,0.201701,0.199988,0.199361,0.199216,0.199287,0.199467,0.199550,0.199068,0.197309,0.193379,...,0.265864,0.291117,0.307184,0.317417,0.324931,0.330352,0.333138,0.333523,0.333286,0.333291
3,0.063540,0.047583,0.047528,0.068287,0.105639,0.149381,0.189406,0.220453,0.242844,0.259747,...,0.333357,0.295564,0.233128,0.175768,0.186533,0.317931,0.557534,0.806746,0.931549,0.851948
4,0.288608,0.293570,0.309065,0.335787,0.364229,0.381273,0.380666,0.368408,0.356563,0.348909,...,0.148214,0.194006,0.233590,0.263993,0.284785,0.298143,0.308815,0.317960,0.325064,0.328412


In [ ]:
# # prompt: plot ppg and ecg data
# # # prompt: plot ppg and ecg data
# # import matplotlib.pyplot as plt
# # # Assuming ppg_data and ecg_data are pandas DataFrames as in your provided code
# # # and contain columns representing time and signal values.  Replace 'PPG' and 'ECG'
# # # with the actual column names if they are different.
# # # Plot PPG data
# # plt.figure(figsize=(12, 6))
# # plt.plot(ppg_data.index, ppg_data['0.281842307'], label='PPG')  # Assuming 'PPG' is the column name for PPG data
# # plt.xlabel('Time')
# # plt.ylabel('Amplitude')
# # plt.title('PPG Signal')
# # plt.legend()
# # plt.grid(True)
# # plt.show()
# # # Plot ECG data
# # plt.figure(figsize=(12, 6))
# # plt.plot(ecg_data.index, ecg_data['0.01997061'], label='ECG') # Assuming 'ECG' is the column name for ECG data
# # plt.xlabel('Time')
# # plt.ylabel('Amplitude')
# # plt.title('ECG Signal')
# # plt.legend()
# # plt.grid(True)
# # plt.show()
# # generate a live plotting from given data

# import matplotlib.pyplot as plt
# import numpy as np

# # Assuming ppg_data and ecg_data are pandas DataFrames as in your provided code
# # and contain columns representing time and signal values.  Replace 'PPG' and 'ECG'
# # with the actual column names if they are different.

# # Create a figure and axes for the plot
# fig, ax1 = plt.subplots(figsize=(12, 6))

# # Plot PPG data on the first y-axis
# color = 'tab:red'
# ax1.set_xlabel('Time')
# ax1.set_ylabel('PPG Amplitude', color=color)
# ax1.plot(ppg_data.index, ppg_data['0.281842307'], color=color, label='PPG') # Replace '0.281842307' with actual column name
# ax1.tick_params(axis='y', labelcolor=color)
# ax1.legend(loc='upper left')


# # Create a second y-axis for ECG data
# ax2 = ax1.twinx()
# color = 'tab:blue'
# ax2.set_ylabel('ECG Amplitude', color=color)
# ax2.plot(ecg_data.index, ecg_data['0.01997061'], color=color, label='ECG') # Replace '0.01997061' with actual column name
# ax2.tick_params(axis='y', labelcolor=color)
# ax2.legend(loc='upper right')

# # Add title and grid
# plt.title('PPG and ECG Signals')
# plt.grid(True)

# # Display the plot
# plt.show()


In [ ]:
sbp_data = pd.read_csv("/content/BTP-6th sem/SBPn.csv")
dbp_data = pd.read_csv("/content/BTP-6th sem/DBPn.csv")

In [ ]:
X_ppg = ppg_data.values
X_ecg = ecg_data.values

In [ ]:
# prompt: convert X_ppg to 1d array

X_ppg_1 = X_ppg.reshape(-1)


In [ ]:
X_ppg.shape

(999, 1024)

In [ ]:
ppg_time_features = extract_time_domain_features(X_ppg)
ppg_freq_features = extract_frequency_domain_features(X_ppg)
ppg_specific_features = extract_ppg_specific_features(X_ppg)
ppg_waveform_features=extract_ppg_features(X_ppg)

In [ ]:
ecg_time_features = extract_time_domain_features(X_ecg)
ecg_freq_features = extract_frequency_domain_features(X_ecg)
ecg_specific_features = extract_ecg_specific_features(X_ecg)

In [ ]:
ppg_time_features

array([[ 0.39613729,  0.31031984,  1.        , ...,  0.49955019,
        -1.24030832,  6.60937528],
       [ 0.38971435,  0.30747815,  1.        , ...,  0.54325685,
        -1.18475375,  6.60848406],
       [ 0.29935053,  0.2109629 ,  1.        , ...,  0.59085199,
        -0.68415382,  6.6763004 ],
       ...,
       [ 0.4371721 ,  0.28190346,  1.        , ...,  0.13152268,
        -1.30567925,  6.69515721],
       [ 0.46637059,  0.29128002,  1.        , ...,  0.09582774,
        -1.37531515,  6.71179275],
       [ 0.49010012,  0.28219603,  1.        , ...,  0.14106384,
        -1.34362551,  6.75172636]])

In [ ]:
# Handle NaN values in the extracted features
#features_df.fillna(features_df.mean(), inplace=True)

In [ ]:
ppg_features = np.hstack((ppg_time_features, ppg_freq_features, ppg_specific_features,ppg_waveform_features))
ecg_features = np.hstack((ecg_time_features, ecg_freq_features, ecg_specific_features))


In [ ]:
features = np.hstack((ppg_features, ecg_features))

In [ ]:
features_df = pd.DataFrame(features, columns=[
    'PPG_Mean', 'PPG_Std', 'PPG_Max', 'PPG_Min', 'PPG_Range', 'PPG_Skewness', 'PPG_Kurtosis', 'PPG_Entropy',
    'PPG_DominantFreq', 'PPG_Power', 'PPG_PulseInterval', 'PPG_PulseAmplitude', 'ETR', 'BD', 'HR', 'SVC', 'DVC',
            'systolic_max_volume_change', 'diastolic_max_volume_change',
            'pulse_wave_intensity_ratio', 'pulse_wave_arrival_time',
      'ECG_Mean', 'ECG_Std', 'ECG_Max', 'ECG_Min', 'ECG_Range', 'ECG_Skewness', 'ECG_Kurtosis', 'ECG_Entropy',
    'ECG_DominantFreq', 'ECG_Power', 'ECG_RRInterval', 'ECG_HRV'
])


In [ ]:
features_df

,PPG_Mean,PPG_Std,PPG_Max,PPG_Min,PPG_Range,PPG_Skewness,PPG_Kurtosis,PPG_Entropy,PPG_DominantFreq,PPG_Power,...,ECG_Max,ECG_Min,ECG_Range,ECG_Skewness,ECG_Kurtosis,ECG_Entropy,ECG_DominantFreq,ECG_Power,ECG_RRInterval,ECG_HRV
0,0.396137,0.310320,1.0,0.000000,1.000000,0.499550,-1.240308,6.609375,1.953125,0.199976,...,1.0,0.000000,1.000000,1.677246,3.749930,6.732841,1.953125,0.063857,0.913000,0.121618
1,0.389714,0.307478,1.0,0.000000,1.000000,0.543257,-1.184754,6.608484,1.953125,0.193921,...,1.0,0.000000,1.000000,1.635673,3.600730,6.737924,1.953125,0.069132,0.904000,0.107852
2,0.299351,0.210963,1.0,0.000000,1.000000,0.590852,-0.684154,6.676300,1.953125,0.093273,...,1.0,0.000000,1.000000,1.736378,4.157087,6.738503,1.953125,0.064792,0.930000,0.122098
3,0.411322,0.312790,1.0,0.025660,0.974340,0.599357,-1.155290,6.641900,1.953125,0.204880,...,1.0,0.000000,1.000000,1.694473,3.953114,6.746027,1.953125,0.063198,0.875556,0.145088
4,0.393073,0.318566,1.0,0.000000,1.000000,0.556385,-1.186942,6.590247,1.953125,0.207496,...,1.0,0.000000,1.000000,1.696087,4.060287,6.751572,1.953125,0.066428,0.871111,0.159441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.432238,0.286832,1.0,0.000000,1.000000,0.195688,-1.309364,6.685102,1.953125,0.161159,...,1.0,0.000000,1.000000,-0.933766,1.888948,6.874804,1.953125,0.070177,0.842667,0.182311
995,0.421311,0.276018,1.0,0.000000,1.000000,0.175761,-1.265022,6.689090,1.953125,0.161602,...,1.0,0.000000,1.000000,-0.913933,1.912000,6.873371,1.953125,0.070537,0.945000,0.175792
996,0.437172,0.281903,1.0,0.000000,1.000000,0.131523,-1.305679,6.695157,1.953125,0.168306,...,1.0,0.000000,1.000000,-1.008943,1.797186,6.869317,1.953125,0.070625,0.895000,0.168781
997,0.466371,0.291280,1.0,0.000000,1.000000,0.095828,-1.375315,6.711793,1.953125,0.174977,...,1.0,0.028914,0.971086,-0.992152,1.941072,6.879230,1.953125,0.067847,0.846222,0.174000


In [ ]:

features_df['SBP'] = sbp_data.values
features_df['DBP'] = dbp_data.values


X = features_df.drop(columns=['SBP', 'DBP'])
y_sbp = features_df['SBP']
y_dbp = features_df['DBP']

X

,PPG_Mean,PPG_Std,PPG_Max,PPG_Min,PPG_Range,PPG_Skewness,PPG_Kurtosis,PPG_Entropy,PPG_DominantFreq,PPG_Power,...,ECG_Max,ECG_Min,ECG_Range,ECG_Skewness,ECG_Kurtosis,ECG_Entropy,ECG_DominantFreq,ECG_Power,ECG_RRInterval,ECG_HRV
0,0.396137,0.310320,1.0,0.000000,1.000000,0.499550,-1.240308,6.609375,1.953125,0.199976,...,1.0,0.000000,1.000000,1.677246,3.749930,6.732841,1.953125,0.063857,0.913000,0.121618
1,0.389714,0.307478,1.0,0.000000,1.000000,0.543257,-1.184754,6.608484,1.953125,0.193921,...,1.0,0.000000,1.000000,1.635673,3.600730,6.737924,1.953125,0.069132,0.904000,0.107852
2,0.299351,0.210963,1.0,0.000000,1.000000,0.590852,-0.684154,6.676300,1.953125,0.093273,...,1.0,0.000000,1.000000,1.736378,4.157087,6.738503,1.953125,0.064792,0.930000,0.122098
3,0.411322,0.312790,1.0,0.025660,0.974340,0.599357,-1.155290,6.641900,1.953125,0.204880,...,1.0,0.000000,1.000000,1.694473,3.953114,6.746027,1.953125,0.063198,0.875556,0.145088
4,0.393073,0.318566,1.0,0.000000,1.000000,0.556385,-1.186942,6.590247,1.953125,0.207496,...,1.0,0.000000,1.000000,1.696087,4.060287,6.751572,1.953125,0.066428,0.871111,0.159441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.432238,0.286832,1.0,0.000000,1.000000,0.195688,-1.309364,6.685102,1.953125,0.161159,...,1.0,0.000000,1.000000,-0.933766,1.888948,6.874804,1.953125,0.070177,0.842667,0.182311
995,0.421311,0.276018,1.0,0.000000,1.000000,0.175761,-1.265022,6.689090,1.953125,0.161602,...,1.0,0.000000,1.000000,-0.913933,1.912000,6.873371,1.953125,0.070537,0.945000,0.175792
996,0.437172,0.281903,1.0,0.000000,1.000000,0.131523,-1.305679,6.695157,1.953125,0.168306,...,1.0,0.000000,1.000000,-1.008943,1.797186,6.869317,1.953125,0.070625,0.895000,0.168781
997,0.466371,0.291280,1.0,0.000000,1.000000,0.095828,-1.375315,6.711793,1.953125,0.174977,...,1.0,0.028914,0.971086,-0.992152,1.941072,6.879230,1.953125,0.067847,0.846222,0.174000


In [ ]:
X

,PPG_Mean,PPG_Std,PPG_Max,PPG_Min,PPG_Range,PPG_Skewness,PPG_Kurtosis,PPG_Entropy,PPG_DominantFreq,PPG_Power,...,ECG_Max,ECG_Min,ECG_Range,ECG_Skewness,ECG_Kurtosis,ECG_Entropy,ECG_DominantFreq,ECG_Power,ECG_RRInterval,ECG_HRV
0,0.396137,0.310320,1.0,0.000000,1.000000,0.499550,-1.240308,6.609375,1.953125,0.199976,...,1.0,0.000000,1.000000,1.677246,3.749930,6.732841,1.953125,0.063857,0.913000,0.121618
1,0.389714,0.307478,1.0,0.000000,1.000000,0.543257,-1.184754,6.608484,1.953125,0.193921,...,1.0,0.000000,1.000000,1.635673,3.600730,6.737924,1.953125,0.069132,0.904000,0.107852
2,0.299351,0.210963,1.0,0.000000,1.000000,0.590852,-0.684154,6.676300,1.953125,0.093273,...,1.0,0.000000,1.000000,1.736378,4.157087,6.738503,1.953125,0.064792,0.930000,0.122098
3,0.411322,0.312790,1.0,0.025660,0.974340,0.599357,-1.155290,6.641900,1.953125,0.204880,...,1.0,0.000000,1.000000,1.694473,3.953114,6.746027,1.953125,0.063198,0.875556,0.145088
4,0.393073,0.318566,1.0,0.000000,1.000000,0.556385,-1.186942,6.590247,1.953125,0.207496,...,1.0,0.000000,1.000000,1.696087,4.060287,6.751572,1.953125,0.066428,0.871111,0.159441
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
994,0.432238,0.286832,1.0,0.000000,1.000000,0.195688,-1.309364,6.685102,1.953125,0.161159,...,1.0,0.000000,1.000000,-0.933766,1.888948,6.874804,1.953125,0.070177,0.842667,0.182311
995,0.421311,0.276018,1.0,0.000000,1.000000,0.175761,-1.265022,6.689090,1.953125,0.161602,...,1.0,0.000000,1.000000,-0.913933,1.912000,6.873371,1.953125,0.070537,0.945000,0.175792
996,0.437172,0.281903,1.0,0.000000,1.000000,0.131523,-1.305679,6.695157,1.953125,0.168306,...,1.0,0.000000,1.000000,-1.008943,1.797186,6.869317,1.953125,0.070625,0.895000,0.168781
997,0.466371,0.291280,1.0,0.000000,1.000000,0.095828,-1.375315,6.711793,1.953125,0.174977,...,1.0,0.028914,0.971086,-0.992152,1.941072,6.879230,1.953125,0.067847,0.846222,0.174000


In [ ]:
pip install catboost

In [ ]:
!pip install catboost xgboost lightgbm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor, AdaBoostRegressor, ExtraTreesRegressor,HistGradientBoostingRegressor, BaggingRegressor
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.linear_model import Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
# from catboost import CatBoostRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor

In [ ]:
X = np.nan_to_num(X, posinf=np.nan, neginf=np.nan)
# Impute or remove NaN values if necessary
# Example using SimpleImputer:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(strategy='mean')  # Choose an appropriate strategy
X = imputer.fit_transform(X)

In [ ]:
X_train, X_test, y_train_sbp, y_test_sbp, y_train_dbp, y_test_dbp = train_test_split(
    X, y_sbp, y_dbp, test_size=0.2, random_state=42
)

In [ ]:
X_train_scaled= X_train
X_test_scaled= X_test

#MODEL TRAINING

In [ ]:
rfr=RandomForestRegressor(random_state=42)
rfr_s=rfr.fit(X_train_scaled,y_train_sbp)
y_pred_sbp=rfr_s.predict(X_test_scaled)
SD_SBP_RF= np.mean(abs(y_test_sbp-y_pred_sbp))
MAE_SBP_RF= np.std(abs(y_test_sbp-y_pred_sbp))
print("sbp_mae:",MAE_SBP_RF)
print("sbp_STD:",SD_SBP_RF)
rfr_d=rfr.fit(X_train_scaled,y_train_dbp)
y_pred_dbp=rfr_d.predict(X_test_scaled)
MAE_DBP_RF= np.mean(abs(y_test_dbp-y_pred_dbp))
SD_DBP_RF= np.std(abs(y_test_dbp-y_pred_dbp))
print("dbp_mae:",MAE_DBP_RF)
print("dbp_STD:",SD_DBP_RF)

sbp_mae: 4.718422697313661
sbp_STD: 4.133228612931667
dbp_mae: 1.5708571666355864
dbp_STD: 1.8000062792916531


In [ ]:
gbr=GradientBoostingRegressor()
gbr_s=gbr.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_GB=gbr_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_GB))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_GB))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
gbr_d=gbr.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_GB=gbr_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_GB))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_GB))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

sbp_mae: 4.9968463810045805
sbp_STD: 4.368638964108799
dbp_mae: 1.6774520899441452
dbp_STD: 1.8574617489014338


In [ ]:
svr=SVR()
svr_s=svr.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_SV=svr_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_SV))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_SV))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
svr_d=svr.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_SV=svr_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_SV))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_SV))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

sbp_mae: 5.899542461732629
sbp_STD: 9.216507608574558
dbp_mae: 6.456870404040481
dbp_STD: 4.448904036381953


In [ ]:
import numpy as np
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

dtr = DecisionTreeRegressor(random_state=42)
grid_search_sbp = GridSearchCV(dtr, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search_sbp.fit(X_train_scaled, y_train_sbp)
dtr_s = grid_search_sbp.best_estimator_
y_pred_sbp_DT = dtr_s.predict(X_test_scaled)


grid_search_dbp = GridSearchCV(dtr, param_grid, cv=5, scoring='neg_mean_absolute_error')
grid_search_dbp.fit(X_train_scaled, y_train_dbp)
dtr_d = grid_search_dbp.best_estimator_
y_pred_dbp_DT = dtr_d.predict(X_test_scaled)

SD_SBP = np.mean(abs(y_test_sbp - y_pred_sbp_DT))
MAE_SBP = np.std(abs(y_test_sbp - y_pred_sbp_DT))
print("SBP - Best hyperparameters:", grid_search_sbp.best_params_)
print("sbp_mae:", MAE_SBP)
print("sbp_STD:", SD_SBP)

MAE_DBP = np.mean(abs(y_test_dbp - y_pred_dbp_DT))
SD_DBP = np.std(abs(y_test_dbp - y_pred_dbp_DT))
print("DBP - Best hyperparameters:", grid_search_dbp.best_params_)
print("dbp_mae:", MAE_DBP)
print("dbp_STD:", SD_DBP)



In [ ]:
dtr=DecisionTreeRegressor()
dtr_s=dtr.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_DT=dtr_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_DT))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_DT))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
dtr_d=dtr.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_DT=dtr_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_DT))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_DT))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:
knn=KNeighborsRegressor()
knn_s=knn.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_KN=knn_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_KN))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_KN))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
knn_d=knn.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_KN=knn_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_KN))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_KN))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:
ADB=AdaBoostRegressor()
ADB_s=ADB.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_AD=ADB_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_AD))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_AD))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
ADB_d=ADB.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_AD=ADB_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_AD))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_AD))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:
# !pip install --upgrade numpy

In [ ]:
# !pip install --upgrade tensorflow

In [ ]:
# !pip install --upgrade keras

In [ ]:
# import numpy as np
# import tensorflow as tf
# from tensorflow import keras
# from tensorflow.keras import layers, regularizers
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import StandardScaler

# # Simulated example dataset (Replace with real PPG feature dataset)
# np.random.seed(42)
# X = np.random.rand(1000, 12)  # 1000 samples, 12 features
# y_sbp = np.random.randint(90, 140, size=(1000, 1))  # Simulated SBP values
# y_dbp = np.random.randint(60, 90, size=(1000, 1))  # Simulated DBP values

# # Split dataset into train and test sets
# X_train, X_test, y_train_sbp, y_test_sbp = train_test_split(X, y_sbp, test_size=0.2, random_state=42)
# X_train, X_test, y_train_dbp, y_test_dbp = train_test_split(X, y_dbp, test_size=0.2, random_state=42)

# # Normalize the data
# scaler = StandardScaler()
# X_train_scaled = scaler.fit_transform(X_train)
# X_test_scaled = scaler.transform(X_test)

# # Learning rate scheduler
# def lr_scheduler(epoch, lr):
#     return lr * 0.99 if epoch > 5 else lr

# # Define the improved neural network model
# def create_model():
#     model = keras.Sequential([
#         layers.Dense(256, activation='swish', kernel_regularizer=regularizers.l2(0.001), input_shape=(X_train.shape[1],)),
#         layers.BatchNormalization(),
#         layers.Dropout(0.3),

#         layers.Dense(128, activation='swish', kernel_regularizer=regularizers.l2(0.001)),
#         layers.BatchNormalization(),
#         layers.Dropout(0.3),

#         layers.Dense(64, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
#         layers.BatchNormalization(),
#         layers.Dropout(0.2),

#         layers.Dense(32, activation='relu'),
#         layers.Dense(1)  # Output layer for regression
#     ])

#     model.compile(optimizer=keras.optimizers.Adam(learning_rate=0.001),
#                   loss='mse', metrics=['mae'])
#     return model

# # Training settings
# early_stopping = keras.callbacks.EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True)
# lr_callback = keras.callbacks.LearningRateScheduler(lr_scheduler)

# # Train model for SBP
# model_sbp = create_model()
# model_sbp.fit(X_train_scaled, y_train_sbp, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping, lr_callback])

# # Train model for DBP
# model_dbp = create_model()
# model_dbp.fit(X_train_scaled, y_train_dbp, epochs=200, batch_size=64, validation_split=0.2, callbacks=[early_stopping, lr_callback])

# # Evaluate SBP model
# y_pred_sbp = model_sbp.predict(X_test_scaled).squeeze()
# mae_sbp = np.mean(np.abs(y_test_sbp.squeeze() - y_pred_sbp))
# std_sbp = np.std(np.abs(y_test_sbp.squeeze() - y_pred_sbp))
# print(f" SBP - MAE: {mae_sbp:.2f}, STD: {std_sbp:.2f}")

# # Evaluate DBP model
# y_pred_dbp = model_dbp.predict(X_test_scaled).squeeze()
# mae_dbp = np.mean(np.abs(y_test_dbp.squeeze() - y_pred_dbp))
# std_dbp = np.std(np.abs(y_test_dbp.squeeze() - y_pred_dbp))
# print(f" DBP - MAE: {mae_dbp:.2f}, STD: {std_dbp:.2f}")


#Saving Model Weights

In [ ]:
import joblib
etr=ExtraTreesRegressor()
etrs= etr.fit(X_train_scaled,y_train_sbp)

joblib.dump(etrs, "e-p-sys.joblib")

y_pred_sbp_ET=etrs.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_ET))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_ET))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
etrd=etr.fit(X_train_scaled,y_train_dbp)

joblib.dump(etrd, "e-p-dys.joblib")

y_pred_dbp_ET=etrd.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_ET))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_ET))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

sbp_mae: 4.315358150385947
sbp_STD: 3.714091314744997
dbp_mae: 1.4832383259440025
dbp_STD: 1.7651787560178538


In [ ]:
models = joblib.load("e-p-sys.joblib")
modeld = joblib.load("e-p-dys.joblib")

y_pred_sbp_ET=models.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_ET))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_ET))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)

y_pred_dbp_ET=modeld.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_ET))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_ET))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

print(y_pred_dbp_ET)


In [ ]:
import matplotlib.pyplot as plt


N = 30

algorithms = [
    ("RandomForest", y_pred_sbp, y_pred_dbp),
    ("GradientBoosting", y_pred_sbp_GB, y_pred_dbp_GB),
    ("SVR", y_pred_sbp_SV, y_pred_dbp_SV),
    ("DecisionTree", y_pred_sbp_DT, y_pred_dbp_DT),
    ("KNN", y_pred_sbp_KN, y_pred_dbp_KN),
    ("AdaBoost", y_pred_sbp_AD, y_pred_dbp_AD),
    ("ExtraTrees", y_pred_sbp_ET, y_pred_dbp_ET)
]


fig, axs = plt.subplots(nrows=7, ncols=1, figsize=(12, 28))

for i, (name, pred_sbp, pred_dbp) in enumerate(algorithms):
    axs[i].plot(y_test_sbp[:N], label='Actual SBP', color='black', linestyle='--')
    axs[i].plot(pred_sbp[:N], label=f'Predicted SBP ({name})', color='blue')

    axs[i].plot(y_test_dbp[:N], label='Actual DBP', color='gray', linestyle='--')
    axs[i].plot(pred_dbp[:N], label=f'Predicted DBP ({name})', color='green')

    axs[i].set_title(f'Actual vs Predicted SBP and DBP - {name} (First {N} Samples)')
    axs[i].set_xlabel('Sample')
    axs[i].set_ylabel('BP Value')
    axs[i].legend()
    axs[i].grid(True)

# Adjust layout
plt.tight_layout()
plt.show()


In [ ]:
import joblib
# Load the saved model
etrs = joblib.load("e-p-sys.joblib")
feature_importance_ETR = etrs.feature_importances_

# Sort feature importances in ascending order
sorted_feature_importance = sorted(zip(features_df.drop(columns=['SBP', 'DBP']).columns, feature_importance_ETR), key=lambda x: x[1])

# Print feature importance with feature names in ascending order
for feature_name, importance in sorted_feature_importance:
    print(f'Feature {feature_name}: Importance {importance}')

In [ ]:
feature_importance_ETR = etrd.feature_importances_
# Print feature importance
for i, importance in enumerate(feature_importance_ETR):
    print(f'Feature {i+1}: Importance {importance}')

In [ ]:
rr= Ridge()
rr_s=rr.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_AD=rr_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_AD))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_AD))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
rr_d=rr.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_AD=rr_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_AD))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_AD))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:
lr= Lasso()
lr_s=lr.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_lr=lr_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_lr))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_lr))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
lr_d=lr.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_lr=lr_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_lr))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_lr))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:
hgb= HistGradientBoostingRegressor()
hgb_s=hgb.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_hgb=hgb_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_hgb))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_hgb))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
hgb_d=hgb.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_hgb=hgb_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_hgb))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_hgb))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:
xgb= XGBRegressor()
xgb_s=xgb.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_xgb=xgb_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_xgb))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_xgb))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
xgb_d=xgb.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_xgb=xgb_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_xgb))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_xgb))
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:
# from catboost import CatBoostRegressor
# cbr= CatBoostRegressor()
# cbr_s=cbr.fit(X_train_scaled,y_train_sbp)
# y_pred_sbp_cbr=cbr_s.predict(X_test_scaled)
# SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_xgb))
# MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_xgb))

# cbr_d=cbr.fit(X_train_scaled,y_train_dbp)
# y_pred_dbp_cbr=cbr_d.predict(X_test_scaled)
# MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_cbr))
# SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_cbr))
# print("sbp_mae:",MAE_SBP)
# print("sbp_STD:",SD_SBP)
# print("dbp_mae:",MAE_DBP)
# print("dbp_STD:",SD_DBP)

In [ ]:
from sklearn.ensemble import BaggingRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_absolute_error
import numpy as np

bagging_regressor = BaggingRegressor(estimator=DecisionTreeRegressor(), n_estimators=100, random_state=42)

# SBP Prediction
sm = bagging_regressor.fit(X_train_scaled, y_train_sbp)
y_pred_sbp_BG = sm.predict(X_test_scaled)
mae_sbp_BG = mean_absolute_error(y_test_sbp, y_pred_sbp_BG)
print("MAE SBP for BG :", mae_sbp_BG)
sbp_std_BG = np.std(abs(y_test_sbp - y_pred_sbp_BG))
print("SD SBP for BG :", sbp_std_BG)

# DBP Prediction
dm = bagging_regressor.fit(X_train_scaled, y_train_dbp)
y_pred_dbp_BG = dm.predict(X_test_scaled)
mae_dbp_BG = mean_absolute_error(y_test_dbp, y_pred_dbp_BG)
print("MAE DBP for BG :", mae_dbp_BG)
dbp_std_BG = np.std(abs(y_test_dbp - y_pred_dbp_BG))
print("SD DBP for BG :", dbp_std_BG)


In [ ]:
pip install lightgbm

In [ ]:
import lightgbm as lgb
lgbr = lgb.LGBMRegressor()
lgbr_s=lgbr.fit(X_train_scaled,y_train_sbp)
y_pred_sbp_lgbr=lgbr_s.predict(X_test_scaled)
SD_SBP= np.mean(abs(y_test_sbp-y_pred_sbp_lgbr))
MAE_SBP= np.std(abs(y_test_sbp-y_pred_sbp_lgbr))

lgbr_d=lgbr.fit(X_train_scaled,y_train_dbp)
y_pred_dbp_lgbr=lgbr_d.predict(X_test_scaled)
MAE_DBP= np.mean(abs(y_test_dbp-y_pred_dbp_lgbr))
SD_DBP= np.std(abs(y_test_dbp-y_pred_dbp_lgbr))
print("sbp_mae:",MAE_SBP)
print("sbp_STD:",SD_SBP)
print("dbp_mae:",MAE_DBP)
print("dbp_STD:",SD_DBP)

In [ ]:


import matplotlib.pyplot as plt
import os

# Number of initial points to visualize
N = 30

new_algorithms = [
    ("LightGBM", y_pred_sbp_lgbr, y_pred_dbp_lgbr),
    ("XGBoost", y_pred_sbp_xgb, y_pred_dbp_xgb),
    ("HistGradientBoosting", y_pred_sbp_hgb, y_pred_dbp_hgb),
    ("Bagging", y_pred_sbp_BG, y_pred_dbp_BG),
    ("GradientBoostingV2", y_pred_sbp_GB, y_pred_dbp_GB)  # Using y_pred_sbp_GB as a placeholder
]
# Create output directory
save_dir = "bp_predictions"
os.makedirs(save_dir, exist_ok=True)

# Loop and save plots
for name, pred_sbp, pred_dbp in new_algorithms:
    plt.figure(figsize=(10, 5))
    #SBP
    plt.plot(y_test_sbp[:N], label='Actual SBP', color='black', linestyle='--')
    plt.plot(pred_sbp[:N], label=f'Predicted SBP ({name})', color='blue')
    #DBP
    plt.plot(y_test_dbp[:N], label='Actual DBP', color='gray', linestyle='--')
    plt.plot(pred_dbp[:N], label=f'Predicted DBP ({name})', color='green')
    plt.title(f'Actual vs Predicted SBP and DBP - {name} (First {N} Samples)')
    plt.xlabel('Sample')
    plt.ylabel('BP Value')
    plt.legend()
    plt.grid(True)
    # Save plot
    filename = os.path.join(save_dir, f"{name}_BP_Prediction.png")
    plt.savefig(filename)
    plt.show() # Display the plot
    plt.close()


In [ ]:
# import matplotlib.pyplot as plt
# import os

# # Number of initial points to visualize
# N = 30

# # Define only new algorithms
# new_algorithms = [
#     ("LightGBM", y_pred_sbp_LGB, y_pred_dbp_LGB),
#     ("XGBoost", y_pred_sbp_XGB, y_pred_dbp_XGB),
#     ("HistGradientBoosting", y_pred_sbp_HGB, y_pred_dbp_HGB),
#     ("Bagging", y_pred_sbp_BAG, y_pred_dbp_BAG),
#     ("GradientBoostingV2", y_pred_sbp_GBM2, y_pred_dbp_GBM2)  # Optional if it's a second GBM
# ]

# # Create output directory
# save_dir = "bp_predictions"
# os.makedirs(save_dir, exist_ok=True)

# # Loop and save plots
# for name, pred_sbp, pred_dbp in new_algorithms:
#     plt.figure(figsize=(10, 5))

#     # SBP
#     plt.plot(y_test_sbp[:N], label='Actual SBP', color='black', linestyle='--')
#     plt.plot(pred_sbp[:N], label=f'Predicted SBP ({name})', color='blue')

#     # DBP
#     plt.plot(y_test_dbp[:N], label='Actual DBP', color='gray', linestyle='--')
#     plt.plot(pred_dbp[:N], label=f'Predicted DBP ({name})', color='green')

#     plt.title(f'Actual vs Predicted SBP and DBP - {name} (First {N} Samples)')
#     plt.xlabel('Sample')
#     plt.ylabel('BP Value')
#     plt.legend()
#     plt.grid(True)

#     # Save plot
#     filename = os.path.join(save_dir, f"{name}_BP_Prediction.png")
#     plt.savefig(filename)
#     plt.close()


In [ ]:
sbp_predd= y_pred_sbp_hgb.reshape([200,1])
dbp_predd= y_pred_dbp_hgb.reshape([200,1])

In [ ]:
y_test_sbp= np.array(y_test_sbp)
y_test_dbp= np.array(y_test_dbp)

In [ ]:
y_test_sbp= y_test_sbp.reshape([200,1])
y_test_dbp= y_test_dbp.reshape([200,1])

In [ ]:
mae_sbp_array= (np.abs(sbp_predd-y_test_sbp))
mae_dbp_array= (np.abs(dbp_predd-y_test_dbp))

In [ ]:
print(mae_sbp_array)


In [ ]:
print(np.mean(np.abs(mae_sbp_array)))
print(np.min(np.abs(mae_sbp_array)))
print(np.max(np.abs(mae_sbp_array)))
print(np.median(np.abs(mae_sbp_array)))

In [ ]:
print(np.mean(np.abs(mae_dbp_array)))
print(np.min(np.abs(mae_dbp_array)))
print(np.max(np.abs(mae_dbp_array)))
print(np.median(np.abs(mae_dbp_array)))

In [ ]:
print(mae_dbp_array)

#BHS Standard

In [ ]:
############### BHS STANDAR SBP ###########################
count_less_than_5 = np.sum(mae_sbp_array <= 5)
count_less_than_10 = np.sum(mae_sbp_array <= 10)
count_less_than_15 = np.sum(mae_sbp_array <= 15)
less_5_inpercrnt= (count_less_than_5/200)*100
less_10_inpercrnt= (count_less_than_10/200)*100
less_15_inpercrnt= (count_less_than_15/200)*100
print(count_less_than_5)
print(count_less_than_10)
print(count_less_than_15)
print(less_5_inpercrnt)
print(less_10_inpercrnt)
print(less_15_inpercrnt)
if less_5_inpercrnt >= 60:
     print("BHS Standar Pass with Grade A (SBP 5 mmhg)")
elif less_5_inpercrnt >= 50:
    print("BHS Standar Pass  with Grade B (SBP 5 mmhg)")
elif less_5_inpercrnt >= 40:
    print("BHS Standar Pass  with Grade C (SBP 5 mmhg)")
else:
    print("BHS Standar Fail (SBP 5 mmhg)")

if less_10_inpercrnt >= 85:
     print("BHS Standar Pass with Grade A (SBP 10 mmhg)")
elif less_10_inpercrnt >= 75:
    print("BHS Standar Pass  with Grade B (SBP 10 mmhg)")
elif less_10_inpercrnt >= 65:
    print("BHS Standar Pass  with Grade C (SBP 10 mmhg)")
else:
    print("BHS Standar Fail (SBP 10 mmhg)")

if less_15_inpercrnt >= 95:
     print("BHS Standar Pass with Grade A (SBP 15 mmhg)")
elif less_15_inpercrnt >= 90:
    print("BHS Standar Pass  with Grade B (SBP 15 mmhg)")
elif less_15_inpercrnt >= 85:
    print("BHS Standar Pass  with Grade C (SBP 15 mmhg)")
else:
    print("BHS Standar Fail (SBP 15 mmhg)")

In [ ]:
############### BHS STANDAR DBP ###########################
count_less_than_5D = np.sum(mae_dbp_array <= 5)
count_less_than_10D = np.sum(mae_dbp_array <= 10)
count_less_than_15D = np.sum(mae_dbp_array <= 15)
less_5_inpercrntD= (count_less_than_5D/200)*100
less_10_inpercrntD= (count_less_than_10D/200)*100
less_15_inpercrntD= (count_less_than_15D/200)*100
print(count_less_than_5D)
print(count_less_than_10D)
print(count_less_than_15D)
print(less_5_inpercrntD)
print(less_10_inpercrntD)
print(less_15_inpercrntD)

if less_5_inpercrntD >= 60:
     print("BHS Standar Pass with Grade A (DBP 5 mmhg)")
elif less_5_inpercrntD >= 50:
    print("BHS Standar Pass with Grade B (DBP 5 mmhg)")
elif less_5_inpercrntD >= 40:
    print("BHS Standar Pass with Grade C (DBP 5 mmhg)")
else:
    print("BHS Standar Fail (DBP 5 mmhg)")

if less_10_inpercrntD >= 85:
     print("BHS Standar Pass with Grade A (DBP 10 mmhg)")
elif less_10_inpercrntD >= 75:
    print("BHS Standar Pass  with Grade B (DBP 10 mmhg)")
elif less_10_inpercrntD >= 65:
    print("BHS Standar Pass  with Grade C (DBP 10 mmhg)")
else:
    print("BHS Standar Fail (DBP 10 mmhg)")

if less_15_inpercrntD >= 95:
     print("BHS Standar Pass with Grade A (DBP 15 mmhg)")
elif less_15_inpercrntD >= 90:
    print("BHS Standar Pass  with Grade B (DBP 15 mmhg)")
elif less_15_inpercrntD >= 85:
    print("BHS Standar Pass  with Grade C (DBP 15 mmhg)")
else:
    print("BHS Standar Fail (DBP 15 mmhg)")

All passed be Grade A under BHS Standard

In [ ]:
import matplotlib.pyplot as plt
# 3. Residual Plot
residuals = np.abs(y_test_sbp - sbp_predd)

plt.figure(figsize=(8, 6))
#plt.plot(residuals, label='Residuals (y_sbp_test - sbp_predd)', color='red')
plt.scatter(range(len(residuals)),residuals, color='blue')
plt.title('Error Plot (SBP)', fontweight='bold')
plt.xlabel('Sample Index', fontweight='bold')
plt.ylabel('Absolute Error', fontweight='bold')
plt.axhline(0, color='black', linestyle='--')
plt.axhline(5, color='black', linestyle='--')
plt.axhline(10, color='black', linestyle='--')
plt.axhline(15, color='black', linestyle='--')
#plt.axhline(20, color='black', linestyle='--')
plt.legend()
plt.show()
#################################################################
residuals = np.abs(y_test_dbp - dbp_predd)

plt.figure(figsize=(8, 6))
#plt.plot(residuals, label='Residuals (y_dbp_test - dbp_predd)', color='red')
plt.scatter(range(len(residuals)),residuals, color='blue')
plt.title('Error Plot (DBP)', fontweight='bold')
plt.xlabel('Sample Index', fontweight='bold')
plt.ylabel('Absolute Error', fontweight='bold')
plt.axhline(0, color='black', linestyle='--')
plt.axhline(5, color='black', linestyle='--')
plt.axhline(10, color='black', linestyle='--')
plt.axhline(14, color='black', linestyle='--')
plt.legend()
plt.show()


In [ ]:


import numpy as np
import matplotlib.pyplot as plt

residuals_sbp = np.abs(y_test_sbp.flatten() - sbp_predd.flatten())

plt.figure(figsize=(8, 6))
plt.scatter(range(len(residuals_sbp)), residuals_sbp, color='blue', label='Residuals')
plt.title('Residual Plot (SBP)', fontweight='bold')
plt.xlabel('Sample Index', fontweight='bold')
plt.ylabel('Absolute Error', fontweight='bold')
plt.axhline(0, color='black', linestyle='--')
plt.axhline(5, color='red', linestyle='--', label='5 mmHg')
plt.axhline(10, color='orange', linestyle='--', label='10 mmHg')
plt.axhline(15, color='green', linestyle='--', label='15 mmHg')
plt.legend()
plt.show()

# 3. Residual Plot for DBP
residuals_dbp = np.abs(y_test_dbp.flatten() - dbp_predd.flatten())

plt.figure(figsize=(8, 6))
plt.scatter(range(len(residuals_dbp)), residuals_dbp, color='blue', label='Residuals')
plt.title('Residual Plot (DBP)', fontweight='bold')
plt.xlabel('Sample Index', fontweight='bold')
plt.ylabel('Absolute Error', fontweight='bold')
plt.axhline(0, color='black', linestyle='--')
plt.axhline(5, color='red', linestyle='--', label='5 mmHg')
plt.axhline(10, color='orange', linestyle='--', label='10 mmHg')
plt.axhline(15, color='green', linestyle='--', label='15 mmHg')
plt.legend()
plt.show()
